In [1]:
import os
import pandas as pd
import tqdm
import gc

# --- НАСТРОЙКИ ---
PATH_TO_DATA = './train_data/' 
OUTPUT_FILE = 'features_pro.parquet'

# Проверяем файлы
files = sorted([f for f in os.listdir(PATH_TO_DATA) if f.endswith('.pq') or f.endswith('.parquet')])
print(f"📂 Найдено файлов: {len(files)}")

# --- ФУНКЦИЯ: ЧИТАЕМ И АГРЕГИРУЕМ ХИТРО ---
def read_parquet_dataset_from_local(path_to_dataset, start_from=0, num_parts_to_read=2):
    res = []
    
    # 1. Основные колонки (ваши списки)
    cols_max = [
        'pre_loans_total_overdue', 'pre_loans_max_overdue_sum', 
        'is_zero_loans5', 'is_zero_loans530', 'is_zero_loans90', 
        'pre_util', 'pre_over2limit', 'pre_maxover2limit',
        'enc_loans_credit_status' # Добавил статус кредита
    ]
    
    cols_mean = [
        'pre_pterm', 'pre_fterm', 'pre_since_opened', 'pre_since_confirmed',
        'pre_loans_credit_limit', 'pre_loans_next_pay_summ',
        'pre_loans_outstanding', 'pre_loans_credit_cost_rate'
    ]
    
    # Формируем список файлов
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset) if filename.endswith('.pq')])
    
    start_from = max(0, start_from)
    real_num_parts = min(len(dataset_paths), num_parts_to_read)
    chunks = dataset_paths[start_from : start_from + real_num_parts]
    
    print(f"🏗 Обрабатываем {len(chunks)} файлов...")

    for chunk_path in tqdm.tqdm(chunks, desc="Чтение файлов"):
        try:
            # Читаем файл целиком (чтобы найти все колонки)
            chunk = pd.read_parquet(chunk_path)
            
            # --- АВТОМАТИЧЕСКИ ИЩЕМ КОЛОНКИ ПЛАТЕЖЕЙ ---
            # Находим все, что начинается на enc_paym_ (история за N месяцев)
            paym_cols = [c for c in chunk.columns if c.startswith('enc_paym_')]
            
            # Собираем словарь: для обычных колонок считаем max, mean, std
            # Для платежей (paym_cols) считаем только max (худший статус)
            agg_funcs = {}
            for col in cols_max + cols_mean:
                if col in chunk.columns:
                    agg_funcs[col] = ['max', 'mean', 'std']
            
            for col in paym_cols:
                agg_funcs[col] = ['max'] # Берем худший статус за месяц
            
            agg_funcs['rn'] = ['max', 'count'] # rn max = всего кредитов
            
            # Группируем
            features = chunk.groupby('id').agg(agg_funcs)
            
            # Делаем плоские названия (pre_util_max)
            features.columns = ['_'.join(col).strip() for col in features.columns.values]
            features = features.reset_index()
            
            res.append(features)
            
            # Чистим память
            del chunk, features
            gc.collect()
            
        except Exception as e:
            print(f"Ошибка в файле {chunk_path}: {e}")
        
    # --- САМОЕ ВАЖНОЕ ИСПРАВЛЕНИЕ ---
    print("🧩 Объединяем и убираем дубликаты...")
    full_df = pd.concat(res, ignore_index=True)
    
    # Финальная группировка, чтобы склеить клиента, если он попал в разные файлы
    return full_df.groupby('id').mean().reset_index()

# --- ЗАПУСК ---
if __name__ == '__main__':
    # Читаем ВСЕ файлы (num_parts_to_read=12)
    df_features = read_parquet_dataset_from_local(PATH_TO_DATA, num_parts_to_read=12)

    print(f"📊 Итоговый размер: {df_features.shape}")
    
    # Заполняем пропуски нулями
    df_features = df_features.fillna(0)

    print("💾 Сохраняем features_pro.parquet...")
    df_features.to_parquet(OUTPUT_FILE)
    print("✅ Готово! Файл создан корректно.")

📂 Найдено файлов: 12
🏗 Обрабатываем 12 файлов...


Чтение файлов: 100%|██████████| 12/12 [00:22<00:00,  1.86s/it]


🧩 Объединяем и убираем дубликаты...
📊 Итоговый размер: (3000000, 79)
💾 Сохраняем features_pro.parquet...
✅ Готово! Файл создан корректно.
